In [ ]:
using DataFrames
using CSV
using LinearAlgebra
using Plots
using Statistics

In [ ]:
#Q1, Minh
function linreg(x, y, lambda)
    m, n = size(x)
    z = x' * x + lambda * Matrix(I, n, n)  # the regularized matrix1
    lin_reg_param = inv(z) * x' * y
    return lin_reg_param
end


In [ ]:
# Tu Pham
# Questions 2, 3 (linreg)
# Function to calculate predicted responses
function predict(linregparam, x)
    predicted_responses = x * linregparam
    return predicted_responses
end

# Function to calculate R^2
function rsq(predicted_responses, true_responses, mean_prediction)
    predictive_model_error = 0
    for i in eachindex(true_responses)
        predictive_model_error += ((true_responses[i] - predicted_responses[i]) ^ 2)
    end
    baseline_model_error = 0
    for i in eachindex(true_responses)
        baseline_model_error += ((true_responses[i] - mean_prediction) ^ 2)
    end
    r_squared = 1 - (predictive_model_error / baseline_model_error)
    return r_squared
end

In [ ]:
path = "usconsumption.csv"
df = CSV.read(path, DataFrame)

In [ ]:
# Part 2, q123, Minh

# Q1 
filepath = "./usconsumption.csv"
df = CSV.read(filepath, DataFrame)
df[!, :Intercept] = ones(size(df, 1)) # create an Intercept column and fill it with 1s.

print(df)

In [ ]:
# Q2
x = df[:, "Quarter"]
rpce = df[:, " Percent Change Consumption"]
rpdi = df[:, "Percent Change Income"]

plot(x, rpce, xlabel="Quarter", legend=true, label="RPCE")

# Q3
plot!(x, rpdi, xlabel="Quarter", legend=true, label="RPDI")
ylabel!("Rate of Change")

In [ ]:
# Matt Le
# Code to prepare for 4, 5 (timeseries)
time_vector = df[:, "Quarter"]
real_RPCE = df[:, " Percent Change Consumption"]
real_RPDI = df[:, "Percent Change Income"]
intercept = df[:, "Intercept"]


RPDI_intercept_features = hcat(real_RPDI, intercept)
linregparam_with_RPDI_intercept = linreg(RPDI_intercept_features, real_RPCE , 0)

println("Weight for RPDI ", linregparam_with_RPDI_intercept[1])
println("Intercept ", linregparam_with_RPDI_intercept[2])

estimated_RPCE = predict(linregparam_with_RPDI_intercept, RPDI_intercept_features)
println("R squared is: ", rsq(estimated_RPCE, real_RPCE, mean(real_RPCE)))


In [ ]:
# Tu Pham
# Question 6 (timeseries)
# Plot RPCE and estimated RPCE against time
function plot_RPCE(real_RPCE, estimated_RPCE, time_vector)
    plot(time_vector, real_RPCE, seriestype=:scatter, label="True response RPCE")
    plot!(time_vector, estimated_RPCE, label="Estimated RPCE")
    xlabel!("Time")
    ylabel!("RPCE value")
    title!("RPCE and estimated RPCE against time")
end
plot_RPCE(real_RPCE, estimated_RPCE, time_vector)

In [ ]:
#Q7, Minh

# Building a matrix of features from all other features
mat = Matrix(df[:, 3:7])
y = (df[:, 2])
# println(size(mat))

result = linreg(mat, y, 0)

# Printing result
# ? Which one will go first?
println("Weight for RPDI: $(result[1])")
println("Weight for IP: $(result[2])")
println("Weight for PS: $(result[3])")
println("Weight for UR: $(result[4])")
println("Intercept: $(result[5])")


In [ ]:
#8, Matt
estimated_RPCE = predict(result, mat)
println("R squared is: ", rsq(estimated_RPCE, y, mean(y)))

println("R squared is increased from .18 to .76, which means that this model predicts the data more accurately.")




In [ ]:
#11, Matt
function getModelWithDifferentReg(lambda)
    mat = Matrix(df[:, 3:7])
    y = (df[:, 2])


    result = linreg(mat, y, lambda)
    estimated_RPCE = predict(result, mat)
    println("R squared with regularization value ", lambda ," is : ", rsq(estimated_RPCE, y, mean(y)))

end

for lambda in [0, 0.001, 0.01, 0.1, 1]
    getModelWithDifferentReg(lambda)
end

println("The models proposed little to none changes in values to each other")

In [ ]:

function getSequentialModel(i)
    

    features = Matrix(df[205:i,[3,4,5,6,7]])

    real_RPCE = (df[205:i, 2])
    estimated_RPCE = predict(result, features)
    rsq(estimated_RPCE, real_RPCE, mean(real_RPCE))
end


for i in 205:227
    println("The R squared from models build by quarters up to ", df[i, "Quarter"], " is ", getSequentialModel(i))
end